# Prueba de GTP-2

## Descarga e instalación de recursos

Para realizar las pruebas sobre GPT-2 vamos a necesitar una serie de recursos que nos permitan acceder a su funcionalidad.

La biblioteca que utilizaremos para acceder a ellos será la biblioteca Transformers de HuggingFace. Este módulo contiene una serie de funcionalidades que permiten realizar funciones de procesamiento de lenguaje, en nuestro caso concreto, generación de lenguaje natural.

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 11.1 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 880 kB 59.6 MB/s 
     |████████████████████████████████| 6.6 MB 16.1 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=321167d5ca1ce9d600dca2c18cfc0e20e692b1a090eab5deda5e4ffb97cd5e27
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Bibliotecas necesarias

A continuación vamos a importar los módulos necesarios para probar el funcionamiento de GPT-2.

En nuestro caso utilizaremos, el módulo *GPT2LMHeadModel* que implementa el modelo de lenguaje GPT-2 y *GPT2Tokenizer* utilizado para el proceso de tokenización.

In [ ]:
from transformers import GPT2LMHeadModel , GPT2Tokenizer

## Tokenizer

El primer paso para poder probar el modelo es convertir los datos de entrada a una secuencia de identificdores de tokens o *tokens ids* comprensibles por el modelo de lenguaje. Utilizaremos la vertion 'gpt-2' del tokenizador *GPT2Tokenizer* para realizar este proceso.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') 

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

{'input_ids': [40, 1842, 39185], 'attention_mask': [1, 1, 1]}

A continuación, se muestra un ejemplo básico de tokenización. Este proceso devuelve unos identificadores de entreda en representación numérica y una máscara de atención.

In [ ]:
print(f'"I love Transformers" : {tokenizer("I love Transformers")}')

"I love Transformers" : {'input_ids': [40, 1842, 39185], 'attention_mask': [1, 1, 1]}


Vemos como influyen los espacios en la generación de tokens, en el caso de "I" se le asigna el token 40; sin embargo, si utilizamos el token " I", se le asigna el token 314.

In [ ]:
print(f'"I love Transformers" : {tokenizer("I love Transformers")}')
print(f'" I love you" : {tokenizer(" I love you")}')

"I love Transformers" : {'input_ids': [40, 1842, 39185], 'attention_mask': [1, 1, 1]}
" I love you" : {'input_ids': [314, 1842, 345], 'attention_mask': [1, 1, 1]}


Podemos controlar el tipo devuelto por el tokenizador. En el caso de la opción *return_tensors="pt"*, el tokenizador devuelve los *input_ids* y la máscara de atención en un objeto tensor.

In [ ]:
tokenizer(' I love you', return_tensors="pt", add_prefix_space=False)

{'input_ids': tensor([[ 314, 1842,  345]]), 'attention_mask': tensor([[1, 1, 1]])}

## Carga del modelo

Primero descargamos el modelo concreto que vamos a utilizar. En nuestro caso, emplearemos la versión 'gpt-2' del modelo y establecemos como token de padding, el token de padding del tokenizer ya que por defecto no viene.

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2' , pad_token_id = tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

## Resultados

Una vez descargados el modelo y todo el resto de rescursos necesarios, probamos la generación de texto a partir de una secuencia. Dada la oración "What is love?" veremos que genera el modelo.

El primer paso, como se mencionó anteriormente, es la tokenización de la secuencia de entrada. Utilizamos el método *encode* del tokenizador, que devolverá los *input_ids* de la secuencia en forma de tensor si le establecemos las opciones adecuadas.

In [ ]:
sequence = "What is love?"
input_ids = tokenizer.encode(sequence, add_prefix_space=False, return_tensors='pt')

In [ ]:
print('input_ids: '+ str(input_ids))

input_ids: tensor([[2061,  318, 1842,   30]])


A continuación, probamos a decodificar esta secuencia de entrada para comprobar que se ha codificado y decodificado de manera correcta. Como la secuencia original y la secuencia decodificada son similare, se puede proceder a la generación de texto a partir de la entrada original.

In [ ]:
decoded_sequence = tokenizer.decode(input_ids[0])

print('decoded seq: '+ str(decoded_sequence))

decoded seq: What is love?


Hacemos uso del método *generate* del modelo para generar, finalmente, el texto. Recibe como argumentos la secuencia original, a partir de la cual generará la salida y otros argumentos escogidos según las opciones por defecto.

In [ ]:
output = model.generate(input_ids, 
    max_length = 50, 
    num_beams = 5,
    no_repeat_ngram_size  = 2,
    early_stopping = True)

Obtenida la salida, comprobamos los resultados de la generación. En este caso, la salida comprende los *output_ids* de la secuencia total de salid, por lo que procedemos finalmente a decodificarlos y comprobar el resultado final.

La secuencia generada corresponde a un texto coherente y fluido en consonancia con la secuencia de entrada.

In [ ]:
print(output)

tensor([[ 2061,   318,  1842,    30,   198,   198, 18565,   318,   257,  1573,
           326,   468,   587,  1088,   329,   257,   890,   640,    13,   632,
           338,   257,   835,   286,  2282,   366,    40,  1842,   345,    11,
           475,   314,   836,   470,   760,   644,   340,  1724,   284,  1842,
          2130,  2073,   526,   632,   460,  1612,  1997,   422,  1842,   329]])


In [ ]:
result = tokenizer.decode(output[0])

In [ ]:
print(result)

What is love?

Love is a word that has been around for a long time. It's a way of saying "I love you, but I don't know what it means to love someone else." It can mean anything from love for
